In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import Model
from tensorflow.keras.layers import  Activation , Conv2D , BatchNormalization , Add , MaxPool2D , GlobalAveragePooling2D , Dense
print("Version of tensorflow is : ",tf.__version__)
from tensorflow.python.keras.utils.vis_utils import plot_model

Version of tensorflow is :  2.9.2


In [15]:
class IdentityBlock(Model):
    def __init__(self, filters, kernal_size):
        super(IdentityBlock, self ).__init__(name = '')
        self.conv = Conv2D(filters, kernal_size, padding='same')
        self.norm = BatchNormalization()
        self.act  = Activation('relu')
        self.add  = Add()

    def call(self , input):
        x = self.conv(input)
        x = self.norm(x)
        x = self.act(x)

        x = self.conv(x)
        x = self.norm(x)

        x = self.add([x , input])
        x = self.act(x)
        return x

In [16]:
class Resnet(Model):
    def __init__(self, num_classes, activation = 'softmax'):
        super(Resnet, self).__init__(name = "")
        self.conv7 = Conv2D(64, 7, padding='same')
        self.norm  = BatchNormalization()
        self.pool  = MaxPool2D((3,3))
        self.idbl1 = IdentityBlock(64, 3)
        self.idbl2 = IdentityBlock(64, 3)
        self.gpool = GlobalAveragePooling2D()
        self.claasifier = Dense(num_classes, activation = tf.keras.activations.get(activation))

    def call(self , input):
        x = self.conv7(input)
        x = self.norm(x)
        x = self.pool(x)
        x = self.idbl1(x)
        x = self.idbl2(x)
        x = self.gpool(x)
        x = self.claasifier(x)

        return x

In [17]:
resnet_model = Resnet(10)

resnet_model.compile(optimizer='adam',
                     loss = tf.keras.losses.sparse_categorical_crossentropy,
                     metrics=['acc'])

In [18]:
def preprocess(f):
    return tf.cast(f['image'] , 'float32') / 255.0  , f["label"]

dataset = tfds.load(name='mnist' , split=tfds.Split.TRAIN)
dataset = dataset.map(preprocess).batch(32)

Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to ~/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [19]:
history = resnet_model.fit(dataset,
                           epochs=3)

Epoch 1/3
1875/1875 [==============================] - 23s 7ms/step - loss: 0.1638 - acc: 0.9577
Epoch 2/3
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0557 - acc: 0.9840
Epoch 3/3
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0399 - acc: 0.9882


In [20]:
dataset_test = tfds.load(name='mnist' , split=tfds.Split.TEST)
dataset_test = dataset_test.map(preprocess).batch(32)

In [21]:
resnet_model.evaluate(dataset_test)

313/313 [==============================] - 2s 6ms/step - loss: 0.4510 - acc: 0.8401


[0.45097553730010986, 0.8400999903678894]